# Project: cleanning - [candy data]
### this notebook is for educational purpose . in this notebook we will do some cleanning

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">wrangling</a></li>
  
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> i will go through candy data 2017 , this data has :
- Internal_ID
- Q1-Going Out?: Are you actually going trick or treating yourself? "has values `Yes or No`"
- Q2-Gender: It has four different options, `Femal ,Male ,Other ,I'd rather not say`
- Q3-Age: Numerical field
- Q4-Country: Text Field, but users have written their own version of the names. Example, for America, there are entries such as USA, us, US, America so we should consider that while cleanning
- Q5-State/Province: Text Field, but users have written their own version of the names. Same as the country data.
- Q6-Joy Or Despair: All kinds of chocolate bars are the questions with three distinct options to choose from (Joy, Meh, Despair).
- Q7-Joy Other: Text Field to write items not included above that give you JOY. Lots of missing values.
- Q8-Despair Other: Text Field ti write items not included above that give you DESPAIR. Lots of missing values.
- Q9-Other Comments: Text Field. Lots of missing values.
- Q10-Dress: Binary field. Missing values present
- Unnamed: 113
- Q11-Day: Binary Answer Field. Missing values present


# 1- import packages and load data

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


In [3]:
pd.options.mode.chained_assignment = None
df = pd.read_excel("candyhierarchy2017.xlsx", encoding='latin-1')
pd.set_option('display.max_columns', None)

<a id='wrangling'></a>
## Data Wrangling



### General Properties

# 2- show some general properties for the data 

In [4]:
df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we don’t know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | Hershey’s Milk Chocolate,Q6 | Hershey's Kisses,Q6 | Hugs (actual physical hugs),Q6 | Jolly Rancher (bad flavor),Q6 | Jolly Ranchers (good flavor),Q6 | JoyJoy (Mit Iodine!),Q6 | Junior Mints,Q6 | Senior Mints,Q6 | Kale smoothie,Q6 | Kinder Happy Hippo,Q6 | Kit Kat,Q6 | LaffyTaffy,Q6 | LemonHeads,Q6 | Licorice (not black),Q6 | Licorice (yes black),Q6 | Lindt Truffle,Q6 | Lollipops,Q6 | Mars,Q6 | Maynards,Q6 | Mike and Ike,Q6 | Milk Duds,Q6 | Milky Way,Q6 | Regular M&Ms,Q6 | Peanut M&M’s,Q6 | Blue M&M's,Q6 | Red M&M's,Q6 | Green Party M&M's,Q6 | Independent M&M's,Q6 | Abstained from M&M'ing.,Q6 | Minibags of chips,Q6 | Mint Kisses,Q6 | Mint Juleps,Q6 | Mr. Goodbar,Q6 | Necco Wafers,Q6 | Nerds,Q6 | Nestle Crunch,Q6 | Now'n'Laters,Q6 | Peeps,Q6 | Pencils,Q6 | Pixy Stix,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Reese’s Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,JOY,JOY,MEH,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN

In [5]:
df.sample(4)


,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we don’t know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | Hershey’s Milk Chocolate,Q6 | Hershey's Kisses,Q6 | Hugs (actual physical hugs),Q6 | Jolly Rancher (bad flavor),Q6 | Jolly Ranchers (good flavor),Q6 | JoyJoy (Mit Iodine!),Q6 | Junior Mints,Q6 | Senior Mints,Q6 | Kale smoothie,Q6 | Kinder Happy Hippo,Q6 | Kit Kat,Q6 | LaffyTaffy,Q6 | LemonHeads,Q6 | Licorice (not black),Q6 | Licorice (yes black),Q6 | Lindt Truffle,Q6 | Lollipops,Q6 | Mars,Q6 | Maynards,Q6 | Mike and Ike,Q6 | Milk Duds,Q6 | Milky Way,Q6 | Regular M&Ms,Q6 | Peanut M&M’s,Q6 | Blue M&M's,Q6 | Red M&M's,Q6 | Green Party M&M's,Q6 | Independent M&M's,Q6 | Abstained from M&M'ing.,Q6 | Minibags of chips,Q6 | Mint Kisses,Q6 | Mint Juleps,Q6 | Mr. Goodbar,Q6 | Necco Wafers,Q6 | Nerds,Q6 | Nestle Crunch,Q6 | Now'n'Laters,Q6 | Peeps,Q6 | Pencils,Q6 | Pixy Stix,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Reese’s Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,JOY,JOY,MEH,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
1113,90277305,No,Female,22,US,FL,MEH,DESPAIR,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,JOY,DESPAIR,JOY,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,JOY,JOY,DESPAIR,MEH,JOY,NaN,JOY,NaN,NaN,JOY,JOY,DESPAIR,MEH,MEH,MEH,JOY,MEH,JOY,NaN,DESPAIR,MEH,JOY,JOY,MEH,MEH,MEH,MEH,JOY,NaN,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,JOY,NaN,JOY,DESPAI

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


In [7]:
df.dtypes

Internal ID                                                                                 int64
Q1: GOING OUT?                                                                             object
Q2: GENDER                                                                                 object
Q3: AGE                                                                                    object
Q4: COUNTRY                                                                                object
Q5: STATE, PROVINCE, COUNTY, ETC                                                           object
Q6 | 100 Grand Bar                                                                         object
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)     object
Q6 | Any full-sized candy bar                                                              object
Q6 | Black Jacks                                                                           object
Q6 | Bonkers (the ca

In [8]:
df.dtypes.iloc[[*range(7),*range(-12,0)]]

Internal ID                           int64
Q1: GOING OUT?                       object
Q2: GENDER                           object
Q3: AGE                              object
Q4: COUNTRY                          object
Q5: STATE, PROVINCE, COUNTY, ETC     object
Q6 | 100 Grand Bar                   object
Q6 | York Peppermint Patties         object
Q7: JOY OTHER                        object
Q8: DESPAIR OTHER                    object
Q9: OTHER COMMENTS                   object
Q10: DRESS                           object
Unnamed: 113                         object
Q11: DAY                             object
Q12: MEDIA [Daily Dish]             float64
Q12: MEDIA [Science]                float64
Q12: MEDIA [ESPN]                   float64
Q12: MEDIA [Yahoo]                  float64
Click Coordinates (x, y)             object
dtype: object

In [9]:
df.nunique().iloc[[*range(7),*range(-12,0)]]

Internal ID                         2460
Q1: GOING OUT?                         2
Q2: GENDER                             4
Q3: AGE                              106
Q4: COUNTRY                          128
Q5: STATE, PROVINCE, COUNTY, ETC     511
Q6 | 100 Grand Bar                     3
Q6 | York Peppermint Patties           3
Q7: JOY OTHER                        867
Q8: DESPAIR OTHER                    685
Q9: OTHER COMMENTS                   385
Q10: DRESS                             2
Unnamed: 113                           1
Q11: DAY                               2
Q12: MEDIA [Daily Dish]                1
Q12: MEDIA [Science]                   1
Q12: MEDIA [ESPN]                      1
Q12: MEDIA [Yahoo]                     1
Click Coordinates (x, y)             952
dtype: int64

In [10]:
round(df.isnull().sum()/len(df),2)

Internal ID                                                                               0.00
Q1: GOING OUT?                                                                            0.04
Q2: GENDER                                                                                0.02
Q3: AGE                                                                                   0.03
Q4: COUNTRY                                                                               0.03
Q5: STATE, PROVINCE, COUNTY, ETC                                                          0.04
Q6 | 100 Grand Bar                                                                        0.30
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)    0.30
Q6 | Any full-sized candy bar                                                             0.27
Q6 | Black Jacks                                                                          0.39
Q6 | Bonkers (the candy)                          

In [11]:
df.duplicated().sum()

0

# 3- write summary about what you get form the above functions
    tips : this summary can be about : if there is any missing values , if there a wrong data type , if there a wrong range of values , ....

1- dataframe shape is 2460 rows & 120 columns 
2- columns like Q12, unnamed: 113, Q7, Q8 & Q9 have too many missing values
3- there are too many difference in country spelling resulting too many unique values
4- the main Q is Q6 which contains 103 columns out of 120 in our dataframe
5- no duplicates

# cleaning

### 4- drop all the useless columns and  rows with too much NaN , 
> tip : useless columns can be like `Internal ID`and can be also columns with too much NaN 
- rename columns for easy access

In [13]:
unwanted_columns=['Q5: STATE, PROVINCE, COUNTY, ETC',"Unnamed: 113", "Internal ID", "Click Coordinates (x, y)",
                  'Q7: JOY OTHER', 'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS']
df.drop(unwanted_columns, axis = "columns",inplace= True)

In [14]:
df.sample(1)

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we don’t know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | Hershey’s Milk Chocolate,Q6 | Hershey's Kisses,Q6 | Hugs (actual physical hugs),Q6 | Jolly Rancher (bad flavor),Q6 | Jolly Ranchers (good flavor),Q6 | JoyJoy (Mit Iodine!),Q6 | Junior Mints,Q6 | Senior Mints,Q6 | Kale smoothie,Q6 | Kinder Happy Hippo,Q6 | Kit Kat,Q6 | LaffyTaffy,Q6 | LemonHeads,Q6 | Licorice (not black),Q6 | Licorice (yes black),Q6 | Lindt Truffle,Q6 | Lollipops,Q6 | Mars,Q6 | Maynards,Q6 | Mike and Ike,Q6 | Milk Duds,Q6 | Milky Way,Q6 | Regular M&Ms,Q6 | Peanut M&M’s,Q6 | Blue M&M's,Q6 | Red M&M's,Q6 | Green Party M&M's,Q6 | Independent M&M's,Q6 | Abstained from M&M'ing.,Q6 | Minibags of chips,Q6 | Mint Kisses,Q6 | Mint Juleps,Q6 | Mr. Goodbar,Q6 | Necco Wafers,Q6 | Nerds,Q6 | Nestle Crunch,Q6 | Now'n'Laters,Q6 | Peeps,Q6 | Pencils,Q6 | Pixy Stix,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Reese’s Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
372,NaN,Male,69,USA,MEH,MEH,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,MEH,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,DESPAIR,JOY,JOY,JOY,DESPAIR,MEH,JOY,MEH,DESPAIR,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,JOY,JOY,JOY,JOY,JOY,JOY,MEH,JOY,JOY,JOY,JOY,JOY,JOY,JOY,JOY,JOY,DESPAIR,MEH,JOY,JOY,JOY,MEH,JOY,JOY,MEH,MEH,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,JOY,JOY,JOY,MEH,DESPAIR,JOY,JOY,JOY,JOY,JOY,MEH,DESPAIR,JOY,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,Blue and black,Sunday,NaN,1.0,NaN,NaN


# Renaming columns

In [15]:
df.rename(columns = {'Q1: GOING OUT?' : 'Going Out?',
                          'Q2: GENDER' : 'Gender',
                          'Q3: AGE': 'Age',
                          'Q4: COUNTRY' : 'Country',
                          'Q10: DRESS' : 'Dress',
                          'Q11: DAY': 'Day',
                          'Q12: MEDIA [Daily Dish]' : 'DailyDish',
                          'Q12: MEDIA [Science]': 'Science',
                          'Q12: MEDIA [ESPN]' : 'ESPN',
                          'Q12: MEDIA [Yahoo]': 'Yahoo'}, inplace=True)

In [16]:
df.columns = [x for x in df.columns.str.replace("Q6 \| ",'Candy ')]

In [17]:
df.sample(1)

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,DailyDish,Science,ESPN,Yahoo
2085,Yes,Male,49,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Drop useless rows

In [18]:
df.iloc[:,4:-6].columns

Index(['Candy 100 Grand Bar',
       'Candy Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Candy Any full-sized candy bar', 'Candy Black Jacks',
       'Candy Bonkers (the candy)', 'Candy Bonkers (the board game)',
       'Candy Bottle Caps', 'Candy Box'o'Raisins', 'Candy Broken glow stick',
       'Candy Butterfinger',
       ...
       'Candy Three Musketeers', 'Candy Tolberone something or other',
       'Candy Trail Mix', 'Candy Twix',
       'Candy Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Candy Vicodin', 'Candy Whatchamacallit Bars', 'Candy White Bread',
       'Candy Whole Wheat anything', 'Candy York Peppermint Patties'],
      dtype='object', length=103)

In [19]:
df[df.iloc[:,5:-6].isnull().all(axis=1)].index

Int64Index([   0,    2,    6,   10,   18,   19,   24,   25,   30,   35,
            ...
            2437, 2440, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2458],
           dtype='int64', length=657)

In [20]:
df[df.iloc[:,5:-6].isnull().all(axis=1)]

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,DailyDish,Science,ESPN,Yahoo
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Male,49,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,No,Male,53,usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Yes,Male

In [21]:
df = df.drop(df[df.iloc[:,5:-6].isnull().all(axis=1)].index)

In [22]:
df = df.reset_index()
df.drop(['index'] , axis="columns", inplace=True)

In [23]:
df.head(3)

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,DailyDish,Science,ESPN,Yahoo
0,No,Male,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,JOY,JOY,MEH,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
1,No,Male,40,us,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,MEH,MEH,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,JOY,JOY,MEH,JOY,JOY,DESPAIR,JOY,MEH,MEH,DESPAIR,JOY,MEH,JOY,MEH,MEH,MEH,MEH,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
2,No,Male,23,usa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAI

### 5 - clean `going out` column 
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [24]:
df['Going Out?'].unique()

array(['No', 'Yes', nan], dtype=object)

In [25]:
df['Going Out?'].describe()

count     1739
unique       2
top         No
freq      1500
Name: Going Out?, dtype: object

In [26]:
df['Going Out?'].value_counts()

No     1500
Yes     239
Name: Going Out?, dtype: int64

In [27]:
df['Going Out?'].isnull().sum()

64

In [28]:
df['Going Out?'] = df['Going Out?'].fillna("No")

In [29]:
df['Going Out?'].unique()

array(['No', 'Yes'], dtype=object)

In [30]:
df['Going Out?'].value_counts()

No     1564
Yes     239
Name: Going Out?, dtype: int64

### 6 - clean ` gender` column 
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [31]:
df['Gender'].unique()

array(['Male', 'Female', "I'd rather not say", 'Other', nan], dtype=object)

In [32]:
df['Gender'].value_counts()

Male                  1087
Female                 618
I'd rather not say      61
Other                   26
Name: Gender, dtype: int64

In [33]:
df['Gender'].isnull().sum()

11

In [34]:
df['Gender'] = df['Gender'].fillna("I'd rather not say")


In [35]:
df['Gender'] = df['Gender'].str.replace("Other","I'd rather not say")

In [36]:
df['Gender'].value_counts()

Male                  1087
Female                 618
I'd rather not say      98
Name: Gender, dtype: int64

In [37]:
df['Gender'].isnull().sum()

0

In [38]:
df['Gender'].unique()

array(['Male', 'Female', "I'd rather not say"], dtype=object)

### 7 - clean ` age` column 
> tip : note the type of age column , and NaN values
 - try to use `to_numeric` 
 - tip : you should choose which best value to put in place of these NaNs ( mean, median, ....etc)

In [51]:
df['Age'].unique()

array([44. , 40. , 23. ,  nan, 33. , 53. , 56. , 64. , 43. , 37. , 59. ,
       48. , 45. , 49. , 34. , 35. , 38. , 58. , 50. , 47. , 52. , 63. ,
       65. , 41. , 31. , 61. , 46. , 54. , 62. , 29. , 42. , 39. , 36. ,
       32. , 28. , 69. , 67. , 30. , 25. , 22. , 51. , 70. , 24. , 27. ,
       18. , 19. , 57. , 66. , 12. , 60. , 55. , 21. , 11. , 68. , 20. ,
        6. , 10. , 71. , 13. , 26. , 72. ,  9. , 39.4, 74. , 15. ,  8. ,
       75. , 17. ,  7. , 14. , 76. , 16. , 77. ,  4. , 73. ])

In [52]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

In [53]:
df['Age'].unique()

array([44. , 40. , 23. ,  nan, 33. , 53. , 56. , 64. , 43. , 37. , 59. ,
       48. , 45. , 49. , 34. , 35. , 38. , 58. , 50. , 47. , 52. , 63. ,
       65. , 41. , 31. , 61. , 46. , 54. , 62. , 29. , 42. , 39. , 36. ,
       32. , 28. , 69. , 67. , 30. , 25. , 22. , 51. , 70. , 24. , 27. ,
       18. , 19. , 57. , 66. , 12. , 60. , 55. , 21. , 11. , 68. , 20. ,
        6. , 10. , 71. , 13. , 26. , 72. ,  9. , 39.4, 74. , 15. ,  8. ,
       75. , 17. ,  7. , 14. , 76. , 16. , 77. ,  4. , 73. ])

In [54]:
df.Age[df['Age']>80] = np.nan

In [55]:
df['Age'].isnull().sum()

63

In [56]:
Age = df.groupby(['Gender'])['Age'].median()
Age = pd.DataFrame(Age)
Age

,Age
Gender,
Female,40.0
I'd rather not say,39.0
Male,43.0


In [57]:
for i in Age.index:
    df.loc[df['Gender']==i, 'Age'] = df.loc[df['Gender']==i, 'Age'].fillna(Age.loc[i])

In [62]:
df['Age'].isnull().sum()

0

In [60]:
df = df[df['Age'].between(5, 80, inclusive="both")]
df['Age']=df['Age'].astype(int)

In [61]:
df['Age'].unique()

array([44, 40, 23, 33, 53, 56, 64, 43, 37, 59, 48, 45, 49, 34, 35, 38, 58,
       50, 47, 52, 63, 65, 41, 31, 61, 46, 54, 62, 29, 42, 39, 36, 32, 28,
       69, 67, 30, 25, 22, 51, 70, 24, 27, 18, 19, 57, 66, 12, 60, 55, 21,
       11, 68, 20,  6, 10, 71, 13, 26, 72,  9, 74, 15,  8, 75, 17,  7, 14,
       76, 16, 77, 73], dtype=int64)

### 8 - clean ` country` column
> tip : note that country column has country names with different writing ways 
  - use fuzzywuzzy to deal with this problem
  - you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [63]:
df['Country'].unique()


array(['USA ', 'us', 'usa', 'canada', 'Canada', 'Us', 'US', 'Murica',
       'USA', 'uk', 'United States', 'united states', 'Usa',
       'United staes', 'United Kingdom', 'United States of America',
       'UAE', 'United States ', 'England', 'canada ', 'United states',
       'u.s.a.', 'USAUSAUSA', 'america', 'france',
       'United States of America ', 'U.S.A.', 'Mexico', 'Canada ',
       'united states of america', 'US of A', 'UK', 'Unites States',
       'The United States', 'North Carolina ', 'Netherlands', 'Europe',
       'U S', 'u.s.', 'Costa Rica', 'The United States of America',
       'cascadia', 'Greece', 'USA? Hard to tell anymore..', "'merica",
       'Pittsburgh', 'United State', 'France', 'U.S.', 'Can', 'Canae',
       'New York', 'Trumpistan', 'Ireland', 'United Sates', 'Korea',
       'Australia', 'California', 'Japan', 'USa', 'South africa',
       'I pretend to be from Canada, but I am really from the United States.',
       'Usa ', 'Iceland', 'Canada`', 'Scotland

In [64]:
df['Country'].value_counts()

USA                                                                     526
United States                                                           383
usa                                                                     155
Canada                                                                  120
US                                                                       95
Usa                                                                      85
USA                                                                      54
United States of America                                                 45
united states                                                            30
canada                                                                   26
United States                                                            26
us                                                                       24
United states                                                            15
Us          

In [65]:
df['Country'] = df['Country'].str.lower().str.strip()

In [66]:
df['Country'].value_counts()

usa                                                                     822
united states                                                           455
canada                                                                  154
us                                                                      129
united states of america                                                 57
uk                                                                       12
u.s.a.                                                                    8
germany                                                                   7
u.s.                                                                      7
united kingdom                                                            6
america                                                                   5
netherlands                                                               4
japan                                                                     3
france      

In [67]:
us= ['united states', 'merica','us','ussa','u.s.a.','us of a']
for usa in us:
    matches = process.extract(usa, df['Country'], limit = df.shape[0])
    for potential_match in matches:
        if potential_match[1] >= 80:
            df.loc[df['Country'] == potential_match[0],'Country'] = 'usa'

In [68]:
df['Country'].value_counts()

usa                            1509
canada                          154
uk                               12
germany                           7
united kingdom                    6
netherlands                       4
japan                             3
scotland                          3
ireland                           3
france                            3
denmark                           2
switzerland                       2
mexico                            2
canae                             1
cascadia                          1
can                               1
iceland                           1
north carolina                    1
pittsburgh                        1
new york                          1
greece                            1
california                        1
endland                           1
new jersey                        1
canada`                           1
china                             1
europe                            1
trumpistan                  

In [69]:
mapping= {'united kingdom':'uk','murrika':'usa','canada`':'canada','canae':'canada','can':'canada','england':'uk',
          'endland':'uk','north carolina':'usa','california':'usa','new jersey':'usa','cascadia':'usa',
          'new york':'usa','scotland':'uk','ireland':'uk','usa? hard to tell anymore..':'usa'}
df['Country'] = df['Country'].replace(mapping)

In [70]:
df['Country'].value_counts()

usa                     1516
canada                   157
uk                        26
germany                    7
netherlands                4
france                     3
japan                      3
switzerland                2
denmark                    2
mexico                     2
europe                     1
greece                     1
iceland                    1
pittsburgh                 1
china                      1
taiwan                     1
trumpistan                 1
singapore                  1
i don't know anymore       1
uae                        1
korea                      1
narnia                     1
south africa               1
spain                      1
south korea                1
costa rica                 1
Name: Country, dtype: int64

In [71]:
df.Country[~df['Country'].isin(['usa', 'uk','canada'])]='other'

In [72]:
df['Country'].value_counts()

usa       1516
canada     157
other       40
uk          26
Name: Country, dtype: int64

### 9 - clean ` area` column
> tip : area column has the same problem as country columns , but looking at all it's unique values may give us another way to deal with it rather than try to fix it

dropped it because it has many unique values

# 10 - clean Q6 columns¶
        6 has all kinds of chocolate bars and questions with three distinct options to choose from (Joy, Meh, Despair).

In [73]:
df.iloc[:,4:-6].isna().sum().sum()

9343

In [74]:
df.iloc[:,4:-6]=df.iloc[:,4:-6].fillna('MEH') 

In [75]:
df.sample(5)

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,DailyDish,Science,ESPN,Yahoo
1729,No,Female,57,usa,JOY,MEH,MEH,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,JOY,JOY,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,MEH,MEH,JOY,DESPAIR,JOY,MEH,DESPAIR,MEH,JOY,JOY,JOY,MEH,MEH,MEH,MEH,MEH,MEH,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,MEH,MEH,JOY,MEH,MEH,DESPAIR,MEH,MEH,MEH,JOY,MEH,DESPAIR,JOY,JOY,MEH,JOY,MEH,MEH,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN,NaN,NaN,NaN,NaN
1386,Yes,Male,26,usa,JOY,MEH,JOY,MEH,MEH,MEH,DESPAIR,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,JOY,MEH,MEH,MEH,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,JOY,JOY,JOY,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,White and gold,Sunday,NaN,NaN,NaN,1.0
1437,No,Male,44,canada,MEH,DESPAIR,JOY,MEH,MEH,MEH,MEH,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,JOY,JOY,DESPAIR,DESP

In [76]:
df.iloc[:,4:-6].isna().sum().sum()

0

### 11 - clean ` dress` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [77]:
df['Dress'].unique()

array(['White and gold', 'Blue and black', nan], dtype=object)

In [78]:
df['Dress']= df['Dress'].fillna('Not interested') 

In [79]:
df['Dress'].unique()

array(['White and gold', 'Blue and black', 'Not interested'], dtype=object)

In [80]:
df['Dress'].isnull().sum()

0

### 12 - clean ` day` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [82]:
df['Day'].unique()

array(['Sunday', 'Friday', nan], dtype=object)

In [83]:
df['Day'].describe()

count       1676
unique         2
top       Friday
freq        1054
Name: Day, dtype: object

In [84]:
df['Day'].isnull().sum()

63

In [85]:
df['Day'] = df['Day'].fillna('Friday')

In [86]:
df['Day'].isnull().sum()

0

In [87]:
df['Day'].value_counts()

Friday    1117
Sunday     622
Name: Day, dtype: int64

# Media cleaning

In [88]:
df.sample(4)

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,DailyDish,Science,ESPN,Yahoo
202,No,Male,48,usa,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,MEH,JOY,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,JOY,JOY,JOY,JOY,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,JOY,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,MEH,Blue and black,Sunday,NaN,NaN,1.0,NaN
548,No,Male,21,uk,MEH,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,JOY,MEH,MEH,JOY,MEH,MEH,MEH,MEH,MEH,DESPAIR,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,JOY,MEH,DESPAIR,MEH,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,JOY,JOY,MEH,MEH,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,MEH,MEH,MEH,MEH,JOY,MEH,MEH,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,MEH,JOY,JOY,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,MEH,MEH,MEH,JOY,MEH,MEH,MEH,MEH,MEH,MEH,Blue and black,Friday,NaN,1.0,NaN,NaN
451,No,Male,32,usa,JOY,DESPAIR,JOY,DESPAIR,MEH,MEH,JOY,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,MEH,JOY,JOY,MEH,DESPAIR,MEH,DESP

In [89]:
media = df.iloc[:,-4:]
for i in media.columns:
    media.loc[media[i] == 1.0, i] = i
df['Media'] = media[media.columns].apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)
df['Media']=df['Media'].astype('category')
df.drop(['DailyDish', 'Science', 'ESPN', 'Yahoo'] , axis="columns", inplace=True)
df.loc[df['Media'] == '', 'Media'] = df['Media'].mode().iloc[0]

In [91]:
df.sample(4)

,Going Out?,Gender,Age,Country,Candy 100 Grand Bar,Candy Anonymous brown globs that come in black and orange wrappers (a.k.a. Mary Janes),Candy Any full-sized candy bar,Candy Black Jacks,Candy Bonkers (the candy),Candy Bonkers (the board game),Candy Bottle Caps,Candy Box'o'Raisins,Candy Broken glow stick,Candy Butterfinger,Candy Cadbury Creme Eggs,Candy Candy Corn,Candy Candy that is clearly just the stuff given out for free at restaurants,Candy Caramellos,"Candy Cash, or other forms of legal tender",Candy Chardonnay,Candy Chick-o-Sticks (we don’t know what that is),Candy Chiclets,Candy Coffee Crisp,Candy Creepy Religious comics/Chick Tracts,Candy Dental paraphenalia,Candy Dots,Candy Dove Bars,Candy Fuzzy Peaches,Candy Generic Brand Acetaminophen,Candy Glow sticks,Candy Goo Goo Clusters,Candy Good N' Plenty,Candy Gum from baseball cards,Candy Gummy Bears straight up,Candy Hard Candy,Candy Healthy Fruit,Candy Heath Bar,Candy Hershey's Dark Chocolate,Candy Hershey’s Milk Chocolate,Candy Hershey's Kisses,Candy Hugs (actual physical hugs),Candy Jolly Rancher (bad flavor),Candy Jolly Ranchers (good flavor),Candy JoyJoy (Mit Iodine!),Candy Junior Mints,Candy Senior Mints,Candy Kale smoothie,Candy Kinder Happy Hippo,Candy Kit Kat,Candy LaffyTaffy,Candy LemonHeads,Candy Licorice (not black),Candy Licorice (yes black),Candy Lindt Truffle,Candy Lollipops,Candy Mars,Candy Maynards,Candy Mike and Ike,Candy Milk Duds,Candy Milky Way,Candy Regular M&Ms,Candy Peanut M&M’s,Candy Blue M&M's,Candy Red M&M's,Candy Green Party M&M's,Candy Independent M&M's,Candy Abstained from M&M'ing.,Candy Minibags of chips,Candy Mint Kisses,Candy Mint Juleps,Candy Mr. Goodbar,Candy Necco Wafers,Candy Nerds,Candy Nestle Crunch,Candy Now'n'Laters,Candy Peeps,Candy Pencils,Candy Pixy Stix,Candy Real Housewives of Orange County Season 9 Blue-Ray,Candy Reese’s Peanut Butter Cups,Candy Reese's Pieces,Candy Reggie Jackson Bar,Candy Rolos,Candy Sandwich-sized bags filled with BooBerry Crunch,Candy Skittles,Candy Smarties (American),Candy Smarties (Commonwealth),Candy Snickers,Candy Sourpatch Kids (i.e. abominations of nature),Candy Spotted Dick,Candy Starburst,Candy Sweet Tarts,Candy Swedish Fish,Candy Sweetums (a friend to diabetes),Candy Take 5,Candy Tic Tacs,Candy Those odd marshmallow circus peanut things,Candy Three Musketeers,Candy Tolberone something or other,Candy Trail Mix,Candy Twix,"Candy Vials of pure high fructose corn syrup, for main-lining into your vein",Candy Vicodin,Candy Whatchamacallit Bars,Candy White Bread,Candy Whole Wheat anything,Candy York Peppermint Patties,Dress,Day,Media
354,Yes,Male,40,usa,JOY,DESPAIR,JOY,MEH,MEH,MEH,JOY,MEH,MEH,JOY,JOY,JOY,DESPAIR,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,JOY,JOY,JOY,JOY,MEH,MEH,JOY,MEH,MEH,MEH,DESPAIR,MEH,JOY,MEH,JOY,JOY,JOY,JOY,MEH,JOY,MEH,DESPAIR,MEH,JOY,JOY,JOY,JOY,JOY,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,MEH,JOY,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,JOY,JOY,MEH,JOY,MEH,JOY,JOY,MEH,JOY,JOY,DESPAIR,JOY,JOY,MEH,MEH,JOY,MEH,DESPAIR,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,White and gold,Friday,Science
173,No,Male,52,usa,MEH,JOY,JOY,MEH,MEH,MEH,JOY,DESPAIR,DESPAIR,JOY,JOY,MEH,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,MEH,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,JOY,JOY,JOY,JOY,JOY,MEH,MEH,JOY,JOY,MEH,JOY,JOY,JOY,JOY,MEH,DESPAIR,DESPAIR,JOY,JOY,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,JOY,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,JOY,MEH,DESPAIR,JOY,JOY,JOY,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,White and gold,Sunday,Science
536,No,Male,39,usa,MEH,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,MEH,DESPAIR,JOY,JOY,JOY,MEH,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,D

### 13 - check
> after finish cleaning you should make sure your data is clean and that :
- there is no wrong data type
- there is no NaN values 
- all the column are cleaing and ready for the next step `EDA`

### 14 - write a summary about all the cleaning steps 

1- understanding data and detect the important columns 
2- started with dropping the columns with many null values which is difficult to clean (7 columns)
3- renaming the columns to be easier to handle
4- dropping the garbage rows to make data more accurate (nearly 700 row)
5- started to fill the nan values in all columns using statistical ways
